In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../')

## Sampler (идея, цель, примеры использования)

### Идея:

бесконечный источник данных из того же распределения, что и данные пришедшие на валидацию. Реализовываться это может подкапотом как угодно.

### Цель:

у некоторых моделей есть ограничения на *resampling* данных. Данный класс позволяет абстрагироваться от этих тонкостей, предоставляя единый интерфейс, плюс единая обертка над данными.


### Типичные примеры исполльзования

In [3]:
# Данные
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from IPython.display import display


X, y = make_classification(n_samples=1000, n_classes=2,
                           n_features=3, n_informative=2,
                           n_redundant=0, random_state=0)
X = pd.DataFrame(X)
y = pd.Series(y)
X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)


display(X_train[:3])
display(X_test[:3])

,0,1,2
906,1.407099,0.936795,-1.307181
145,-0.830887,-0.505341,1.206138
58,2.097963,-0.034475,0.373189


,0,1,2
339,-1.701028,0.731294,0.373625
624,0.447498,1.384187,1.906693
241,1.292023,0.386166,-0.516310


In [4]:
# Создаем sampler
from sbe_vallib import SupervisedSampler

sampler = SupervisedSampler(train={'X': X_train, 'y_true': y_train},
                            oos = {'X': X_test, 'y_true': y_test})

In [5]:
# Получаем resampling версию данных

sampler.set_state(seed=5, gen_method='bootstrap')

display(sampler.train['X'][:4])
display(sampler.oos['X'][:4])


,0,1,2
906,1.407099,0.936795,-1.307181
145,-0.830887,-0.505341,1.206138
58,2.097963,-0.034475,0.373189
259,1.166510,0.633534,-0.116273


,0,1,2
340,0.949110,-0.080599,-0.314469
389,1.379352,-0.767949,-1.822861
451,-1.515180,0.450289,0.989375
362,-0.415896,0.514004,1.741516


In [6]:
# Получаем исходные данные

sampler.reset()

display(sampler.train['X'][:4])
display(sampler.oos['X'][:4])

,0,1,2
906,1.407099,0.936795,-1.307181
145,-0.830887,-0.505341,1.206138
58,2.097963,-0.034475,0.373189
259,1.166510,0.633534,-0.116273


,0,1,2
339,-1.701028,0.731294,0.373625
624,0.447498,1.384187,1.906693
241,1.292023,0.386166,-0.516310
309,0.195713,-0.522579,0.857511


## Scorer (идея, цель, примеры использования)

### Идея:

Идея этого класса - посчитать скопом заранее заданные метрики при вызове метода ```scorer.score()```

### Цель:

Причина появления: модели могут оцениваются самыми разными способами, все не уложить в тип ```func(y_true, y_pred)``` (*например YOLO*). С помощью этого класса абстрагируемся от этого, оставляя реализацию в нестандартных случаях на валидатора.


In [7]:
# создаем scorer
from sbe_vallib import BinaryScorer
from sbe_vallib.validation.utils.metrics import BINARY_METRICS

scorer = BinaryScorer(metrics=BINARY_METRICS, cutoff=0.1)

In [8]:
list(BINARY_METRICS.keys())

['accuracy_score', 'precision_score', 'recall_score', 'f1_score', 'gini']

### Типичные примеры исполльзования

In [9]:
# для начала создадим модель

from sklearn.linear_model import LogisticRegression

sampler.reset()
model = LogisticRegression().fit(sampler.train['X'], sampler.train['y_true'])

In [10]:
scorer.score(sampler.oos['y_true'], model.predict_proba(sampler.oos['X']))

{'accuracy_score': 0.86,
 'precision_score': 0.7872340425531915,
 'recall_score': 0.9866666666666667,
 'f1_score': 0.8757396449704142,
 'gini': 0.9565333333333332}

## Test (идея, цель, примеры использования)

### Идея:

Функция, которая реализует подсчет теста и его оформление. На эту функцию накладываются ограничения на сигнатуру: 

- функция как минимум должна принимать параметры ```model, scorer, sampler, **kwargs``` 
- функция должна выдать результат работы в следующем формате (привет SberDS)
    ```
    {
        "semaphore": str one of {"gray", "green", "yellow", "red"},
        "result_dict": python object,
        "result_dataframes": List[pd.DataFrame],
        "result_plots": List[plt.figure],
    }
    ```
    где,
    - "semaphore" -- светофор выставленный за тест
    - "result_dict" -- python object, который валидатор посчитает полезным для дальнейшего использования
    - "result_dataframes" -- список таблиц, которые будут отражены в агрегированном excel файле
    - "result_plots" -- список рисунков, которые будут отражены в агрегированном excel файле.
- ```sampler``` это единственный источник данных


### Цель:

Провести кастомный тест или из методики


In [11]:
# напишем какой-то тупой тест

def custom_test(model, scorer, sampler, threshold = 1, **kwargs):
    semaphore = 'green'
    if len(sampler.train['X']) < threshold:
        semaphore = 'red'
    
    df_table = pd.DataFrame({'len': [len(sampler.train['X'])],
                             'semaphore': semaphore})
    
    return {
        "semaphore": semaphore,
        "result_dict": {'train_shape':sampler.train['X'].values.shape},
        "result_dataframes": [df_table],
        "result_plots": [],
    }

custom_test(model, scorer, sampler, threshold=5, r_u_angry='True')

{'semaphore': 'green',
 'result_dict': {'train_shape': (700, 3)},
 'result_dataframes': [   len semaphore
  0  700     green],
 'result_plots': []}

## Pipeline (идея, цель, примеры использования)

### Идея:

Отражение понятия методики в нашей бибилиотеке. То есть ```Pipeline``` это список тестов с указанными параметрами и дополнительными метаданными для формирования отчета (*цель теста, интерпретация, способ агрегации блоков*).

### Цель:

Капсулировать всю информацию из методички


## Реализация:

```Pipeline``` - это excel-файл с двумя страницами
- первая страница -- информация о тестах
- вторая страница -- информация о блоках.

In [12]:
pd.read_excel('../sbe_vallib/validation/table/pipelines/Config_31.xlsx', sheet_name="tests_config")

,import_path,test_key,block_key,informative,Название блока,Название,Цель,Интерпретация,Границы красный,Границы желтый,Границы зеленый,params
0,sbe_vallib.validation.table.general_tests.data...,test_factor_psi,data_quality,0,Качество данных,Тест 1.2 Анализ однородности выборок out-of-sa...,Убедиться в однородности выборки (в разрезе фа...,Высокое значение PSI свидетельствует о различи...,NaN,"PSI ≥ 0,2","PSI < 0,2","{""merge_upto_quantile"": 0.05,\n ""rounding_prec..."
1,sbe_vallib.validation.table.general_tests.mode...,test_ci,model_quality,0,Качество модели,Тест 2.2 Доверительный интервал ключевой метри...,Наглядно изобразить уровень статистической пог...,Данный тест позволяет оценить фактический разб...,NaN,NaN,NaN,"{""n_iter"": 200, ""use_predict_proba"": True}"


In [13]:
pd.read_excel('../sbe_vallib/validation/table/pipelines/Config_31.xlsx', sheet_name="agg_config")

,block_key,func
0,data_quality,sbe_vallib.validation.worst_semaphore
1,model_quality,sbe_vallib.validation.worst_semaphore
2,calibration,sbe_vallib.validation.worst_semaphore
3,specification,sbe_vallib.validation.worst_semaphore
4,stability,sbe_vallib.validation.worst_semaphore


## Validation (идея, цель, примеры использования)

### Идея:

Запуск всех тестов из списка и аггрегация их результатов.

### Цель:

Удобный запуск тестов


<img src="./images/uml_vallib.jpg" alt="fishy" class="bg-primary mb-1" width="500px">

In [14]:
# способ прокидывания custim_test'ов в validation

from sbe_vallib import Validation

custom_tests = {
    "custom_1": {"block": "data_quality", "callable": custom_test, "params": {"threshold": 5000}}
}


validor = Validation(model,
                     sampler,
                     scorer,
                     custom_tests=custom_tests,
                     pipeline='../sbe_vallib/validation/table/pipelines/Config_31.xlsx')
res = validor.validate()

In [15]:
list(res.keys())

['test_factor_psi', 'test_ci', 'custom_1']

In [18]:
res['custom_1']['result_dataframes'][0]

,len,semaphore
0,700,red
